In [1]:
import math
import numpy as np
from sympy import *
x=Symbol('x') # x1
t=Symbol('t')
from scipy.optimize import fsolve

d-digit chopping:

In [ ]:
def chop(value, d=3): # default is 3-digit chopping
    if value==0:
        return 0
    v=abs(value)
    xx = str(v)
    m = '{:.'+str(d+1)+'e}'
    y = m.format(v)
    y1=y.split("e")[0]
    yy=float(y1[0:d+1])
    zz=int(v/float(y1))
    z0=round(float(y1)/v)
    if v<1 and -1<v:
        result= float(yy/z0)
    else:
        if zz==0: # handle the case of irrationals
            result=yy
        else:
            result= float(yy*zz)
    if value<0:
        return -result
    else:
        return result

Gaussian Elimination with Backward Substitution:

In [4]:
def findP(i,n,A):
    
    for k in range(i,n+1):
        if A[k][i]!=0:
            return k
    return -1 # means p cannot be found

def GEBack(n,A): # set A[0][k]=0, A[k][0]=0 for convinience
    '''
    n: number of unknowns and equations
    A: augmented matrix
    '''
    for i in range(1,n):
        p=findP(i,n,A)
        if p==-1:
            print(" no unique solution exists")
            return
        if p!= i:
            temp=A[i]
            for k in range(1,n+2):
                A[i][k]=A[p][k]
                A[p][k]=temp[k]
        for j in range(i+1,n+1):
            m=A[j][i]/A[i][i]
            for k in range(0,n+2):
                A[j][k]=A[j][k]-m*A[i][k]
    if A[n][n]==0:
        print(" no unique solution exists")
        return
    xx=[0]*(n+1)
    xx[n]=A[n][n+1]/A[n][n]
    for i in range(n-1,0,-1):
        sum=0
        for j in range(i+1,n+1):
            sum+=A[i][j]*xx[j]
        xx[i]=(A[i][n+1]-sum)/A[i][i]
    return xx

Gaussian Elimination with scaled partial pivoting:

In [3]:
def findP2(i,n,NROW,A,s):
    maxV=0
    index=0
    for j in range(i,n+1):
        if abs(A[NROW[j]][i])/s[NROW[j]] > maxV:
            maxV=abs(A[NROW[j]][i])/s[NROW[j]]
            index=j
    return j

def findmax(l):
    maxV=0
    for value in l:
        if value >maxV:
            maxV=value
    return maxV

def GESPP(n,A): # set A[0][k]=0, A[k][0]=0 for convinience
    '''
    n: number of unknowns and equations
    A: augmented matrix
    '''
    NROW=[0]*(n+1)
    s=[0]*(n+1)
    for i in range(1,n+1):
        s[i]=findmax(A[i])
        if s[i]==0:
            print(" no unique solution exists")
            return
        else:
            NROW[i]=i

    for i in range(1,n):
        p=findP2(i,n,NROW,A,s)
        if A[NROW[p]][i]==0:
            print(" no unique solution exists")
            return
        if NROW[i]!= NROW[p]:
            NCOPY=NROW[i]
            NROW[i]=NROW[p]
            NROW[p]=NCOPY
        for j in range(i+1,n+1):
            m=A[NROW[j]][i]/A[NROW[i]][i]
            for k in range(0,n+2):
                A[NROW[j]][k]=A[NROW[j]][k]-m*A[NROW[i]][k]
    if A[NROW[n]][n]==0:
        print(" no unique solution exists")
        return
    xx=[0]*(n+1)
    xx[n]=A[NROW[n]][n+1]/A[NROW[n]][n]
    for i in range(n-1,0,-1):
        sum=0
        for j in range(i+1,n+1):
            sum+=A[NROW[i]][j]*xx[j]
        xx[i]=(A[NROW[i]][n+1]-sum)/A[NROW[i]][i]
    return xx,A

$LU$ Factorization:

In [ ]:
def LUFac(n,A,LorU): # LorU is a string
    '''
    n: dimension
    A: n*n matrix [a_{ij}] to be factored
    LorU: choose the main diagonal of which matrix between L and U consists of all 1s.
    '''
    U=np.zeros([n,n])
    L=np.zeros([n,n])
    if LorU=='L':
        for i in range(0,n):
            L[i][i]=1
        U[0][0]=A[0][0]/L[1][1]
    if LorU=='U':
        for i in range(0,n):
            U[i][i]=1
        L[0][0]=A[0][0]/U[1][1]
    if L[0][0]*U[0][0]==0:
        print("Factorization impossible")
        return
    for j in range(1,n):
        U[0][j]=A[0][j]/L[0][0] # first row of U
        L[j][0]=A[j][0]/U[0][0] # first column of L
    for i in range(1,n-1):
        sum1=0
        for k in range(0,i):
            sum1+=L[i][k]*U[k][i]
        if LorU=='L':
            U[i][i]=(A[i][i]-sum1)/L[i][i]
        if LorU=='U':
            L[i][i]=(A[i][i]-sum1)/U[i][i]
        if L[i][i]*U[i][i]==0:
            print("Factorization impossible")
            return
        for j in range(i+1,n):
            sum2=0
            sum3=0
            for k in range(0,i):
                sum2+=L[i][k]*U[k][j]
                sum3+=L[j][k]*U[k][i]
            U[i][j]=1/L[i][i]*(A[i][j]-sum2) # ith row of U
            L[j][i]=1/U[i][i]*(A[j][i]-sum3) # ith column of L
    sum4=0
    for k in range(0,n-1):
        sum4+=L[n-1][k]*U[k][n-1]
    if LorU=='L':
        U[n-1][n-1]=(A[n-1][n-1]-sum4)/L[n-1][n-1]
    if LorU=='U':
        L[n-1][n-1]=(A[n-1][n-1]-sum4)/U[n-1][n-1]
    return L,U